### 单头注意力

单头注意力一般使用点积注意力，Scaled Dot-Product Attention

即使用两个向量之间的点积来衡量两个向量 (query 和 key) 之间的相关性

首先对查询向量以及键值对向量进行线性映射，让三个向量都映射到同一个向量空间中（维度相同）

然后再做点积获取注意力分数，再通过 softmax 层进行归一化得到注意力权重，最后应用到各个值得到最终的输出

$$
Q = QW^Q, \quad K = KW^K, \quad V = VW^V
$$

$$
\text{Attention}(X) = 
\operatorname{softmax}\!\left(\frac{QK^\top}{\sqrt{d_k}}\right) V
$$

$$
其中 d_k 表示进行线性映射后 Q、K、V 的向量维度
$$

$$
除\sqrt{d_k}的作用是防止向量点积之间的方差过大，导致softmax饱和
$$

单头注意力图解如下：

![](md-img\单头注意力.jpg)

<br>

### 代码实现

In [ ]:
import torch
from torch import nn

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, query_size, key_size, value_size, model_size):
        super().__init__()
        self.model_size = torch.tensor(model_size)
        self.linear_q = nn.Linear(query_size, model_size)
        self.linear_k = nn.Linear(key_size, model_size)
        self.linear_v = nn.Linear(value_size, model_size)
        self.softmax = nn.Softmax(dim=2)

    # query：(batch_size, num_querys, query_size)
    # key：(batch_size, num_pairs, key_size)
    # value：(batch_size, num_pairs, value_size)
    def forward(self, query, key, value):
        query = self.linear_q(query)    # (batch_size, num_querys, model_size)
        key = self.linear_k(key)        # (batch_size, num_pairs, model_size)
        value = self.linear_v(value)    # (batch_size, num_pairs, model_size)
        score = torch.bmm(query, key.permute(0, 2, 1))    # (batch_size, num_querys, num_pairs)
        score = score / torch.sqrt(self.model_size)       # (batch_size, num_querys, num_pairs)，scale
        weight = self.softmax(score)    # (batch_size, num_querys, num_pairs)
        return torch.bmm(weight, value) # (batch_size, num_querys, model_size)